# Logistic Regrssion - Multinomial

In [1]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
ds_train = datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
train_loader = torch.utils.data.DataLoader(ds_train,batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=32, shuffle=True)

In [3]:
x,y = next(iter(train_loader))
x.shape, y.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

In [4]:
class LogisticRegression(nn.Module):
    
    def __init__(self, in_features, out_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(in_features, out_classes)
        
    def forward(self, x):
        bs = x.shape[0]
        x = x.view(bs, -1) #flatten
        x = self.linear(x)
        x = F.log_softmax(x, 1)
        return x
        

In [5]:
model = LogisticRegression(28*28, 10)

Training code

In [6]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.NLLLoss()
epochs = 10
model.to('cuda')
for e in range(epochs):
    print(f"start epoch {e}")
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        yhat = model(inputs)
        loss = criterion(yhat, labels)
        loss.backward()
        optimizer.step()
    
    # run validation
    model.eval()
    val_loss = 0.0
    correct = 0.0
    num = 0.0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        with torch.no_grad():
            yhat = model(inputs)
            val_loss += criterion(yhat, labels) * inputs.shape[0]
            _, yl = torch.max(yhat, 1)
            correct += torch.sum(yl == labels).item()
            num += inputs.shape[0]
    print(f"valiation loss is {val_loss/num} , accuracy is {correct/num}")
    

start epoch 0
valiation loss is 0.2912658751010895 , accuracy is 0.9183
start epoch 1
valiation loss is 0.27888596057891846 , accuracy is 0.923
start epoch 2
valiation loss is 0.2824396789073944 , accuracy is 0.9189
start epoch 3
valiation loss is 0.284843772649765 , accuracy is 0.9207
start epoch 4
valiation loss is 0.2886124551296234 , accuracy is 0.9188
start epoch 5
valiation loss is 0.27440404891967773 , accuracy is 0.9266
start epoch 6
valiation loss is 0.31283023953437805 , accuracy is 0.9183
start epoch 7
valiation loss is 0.28688353300094604 , accuracy is 0.9219
start epoch 8
valiation loss is 0.297529399394989 , accuracy is 0.9218
start epoch 9
valiation loss is 0.27903297543525696 , accuracy is 0.9235
